# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 단순 질의용 prompt 생성-from_template() 메소드 이용. (PromptTemplate용)
prompt = ChatPromptTemplate.from_template(
    template="한국의 {topic}에 관련된 속담 {count}개를 알려줘. 목록 형식으로 출력해줘."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()
# prompt -> model -> parser


In [ ]:
query = prompt.invoke({"topic":"호랑이", "count":4})
res = model.invoke(query)
final_res = parser.invoke(res)

In [48]:
chain = prompt | model  | parser
final_res = chain.invoke({"topic":"호랑이", "count":4})
# final_res = final_res.invoke(final_res)

In [49]:
print(final_res)

1. 호랑이도 제 말하면 온다 — 어떤 사람을 말하면 그 사람이 뜻밖에 나타난다는 뜻(영어의 "speak of the devil")
2. 호랑이 없는 골에 여우가 왕이다 — 강한 사람이 없을 때 약한 사람이 잘난 체한다는 뜻
3. 호랑이에게 물려가도 정신만 차리면 산다 — 큰 위기에서도 침착하면 살아남을 수 있다는 뜻
4. 호랑이 굴에 들어가야 호랑이 새끼를 잡는다 — 큰 성과는 위험을 감수해야 얻을 수 있다는 뜻


In [43]:
print(final_res) # res에서 content값만 뽑아낸 거.

first=ChatPromptTemplate(input_variables=['count', 'topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['count', 'topic'], input_types={}, partial_variables={}, template='한국의 {topic}에 관련된 속담 {count}개를 알려줘. 목록 형식으로 출력해줘.'), additional_kwargs={})]) middle=[ChatOpenAI(profile={'max_input_tokens': 400000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000001AF9E53D880>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AF9DE7AF00>, root_client=<openai.OpenAI object at 0x000001AF9E53C110

## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [22]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
txt = "이순신,유관순,안중근,강감찬,세종대왕"
txt = "요청하신 이름은 이순신과 유관순과 안중근입니다."
r1 = parser.parse(txt)
r2 = parser.parse(txt)
print(type(r1), type(r2))

<class 'list'> <class 'list'>


In [23]:
print(r1)
print(r2)

['요청하신 이름은 이순신과 유관순과 안중근입니다.']
['요청하신 이름은 이순신과 유관순과 안중근입니다.']


In [ ]:
# 이 output parser에 맞는 출력을 모델이 하도록 하는 지시문장으로 프롬프트에 추가한다.
parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [ ]:
parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate(
    messages=[
        {"role":"system", "content": "Output Format: {format_instruction}"},
        {"role":"user", "content":"{subject}에 대해 다섯가지를 나열해 주세요."}
    ],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
# partial_variables ={"input_variable":넣을 값} => input_variable의 값을 Prompt Template을 만들면서 넣어준다.
# (invoke 시 넣는 것이 아니라 생성시 설정.)

model = ChatOpenAI(model="gpt-5-mini")

query = prompt.invoke({"subject":"자동차 종류"})
res = model.invoke(query)
res


AIMessage(content='세단, SUV, 해치백, 쿠페, 픽업트럭', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 282, 'prompt_tokens': 56, 'total_tokens': 338, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cnbfs6UJsfoeYz2iZmKVUyKLoXday', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a29-97e7-79d1-9011-f2b916311498-0', usage_metadata={'input_tokens': 56, 'output_tokens': 282, 'total_tokens': 338, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

In [36]:
print(res.content)
final_res = parser.invoke(res)
final_res

세단, SUV, 해치백, 쿠페, 픽업트럭


['세단', 'SUV', '해치백', '쿠페', '픽업트럭']

In [39]:
# 질의 -> (Prompt) -> query -> (Model) -> res -> (Parser) -> 최종답변
# Chain: 위 실행흐름을 자동화. (pipeline) 같은거.
chain = prompt | model | parser
res = chain.invoke({"subject":"오픈소스 LLM 모델 이름"})
print(res)

['GPT-J', 'GPT-NeoX-20B', 'MPT-7B', 'BLOOM', 'Falcon-40B']


## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [53]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
print(parser.get_format_instructions())

txt = '{"name":"홍길동", "age":20}'
r = parser.invoke(txt)
print(type(r))
r['age']

Return a JSON object.
<class 'dict'>


20

In [56]:
parser = JsonOutputParser()
prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format:{output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables = {"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)
final_res = parser.invoke(res)

In [61]:
print(type(res.content), type(final_res))

<class 'str'> <class 'dict'>


In [ ]:
final_res

{'이름': '이순신',
 '출생년도': 1545,
 '사망년도': 1598,
 '간단소개': '조선 임진왜란(1592–1598) 당시의 대표적 해군 지휘관. 명백한 성과와 불굴의 용기로 조선 수군을 이끌어 일본군의 해상 보급선을 차단·파괴하여 국가적 위기 극복에 기여했다. 사후 충무공(忠武公)이라는 시호를 받음.',
 '주요직책': ['전라좌수사(전라좌수영 수군 지휘관)', '삼도수군통제사(三道水軍統制使, 조선의 해군 통합 지휘관)'],
 '대표업적': ['해전에서의 연이은 승리로 일본군의 해상 보급로를 차단하여 육지 전선의 반격을 가능하게 함',
  '학익진(鶴翼陣) 같은 전술 운용과 기동성 높은 전선 지휘로 열세를 극복함',
  '거북선(龜船) 등 함선 운용으로 해전에서 우위를 확보함(거북선의 설계·운용에 관한 세부 사항은 학자들 사이에 일부 논쟁이 있음)',
  "개인 일기인 '난중일기(亂中日記)'를 통해 전쟁 당시 상황과 지휘 모습을 상세히 남김"],
 '주요전투': [{'이름': '한산도 대첩 (한산도)',
   '연도': 1592,
   '의미': '임진왜란 초기 대표적 대승으로 일본 수군의 기동력을 크게 약화시켜 해상 주도권을 확보함'},
  {'이름': '옥포해전', '연도': 1592, '의미': '임진왜란 초기의 승전으로 조선 수군의 기세를 높임'},
  {'이름': '사천포해전', '연도': 1592, '의미': '해상에서의 연속된 승리로 일본 측 보급과 해상 활동에 타격을 줌'},
  {'이름': '명량해전 (명량)',
   '연도': 1597,
   '의미': '수적 열세(기록에 따라 차이가 있음)에서 조선 수군이 극적인 승리를 거둔 전투로 가장 유명한 일화 중 하나'},
  {'이름': '노량해전 (노량)',
   '연도': 1598,
   '의미': '임진왜란의 사실상 마지막 해전. 이순신이 전사한 전투로 알려져 있음'}],
 '전술특징': '제해권 확보와 기동력을 중시한 전술 운용, 지형·조류·기후를 활용한 작전, 군사·민간 정보

In [65]:
final_res['대표업적']

['해전에서의 연이은 승리로 일본군의 해상 보급로를 차단하여 육지 전선의 반격을 가능하게 함',
 '학익진(鶴翼陣) 같은 전술 운용과 기동성 높은 전선 지휘로 열세를 극복함',
 '거북선(龜船) 등 함선 운용으로 해전에서 우위를 확보함(거북선의 설계·운용에 관한 세부 사항은 학자들 사이에 일부 논쟁이 있음)',
 "개인 일기인 '난중일기(亂中日記)'를 통해 전쟁 당시 상황과 지휘 모습을 상세히 남김"]

In [ ]:
# JSON 응답 스키마(구조 설계) 정의 -> Pydantic
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")
    
# 변수: key
# Field.description:변수에 넣을 내용


In [70]:
parser2 = JsonOutputParser(pydantic_object=PersonInfo)
print(parser2.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [71]:
parser = JsonOutputParser(pydantic_object=PersonInfo)
prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format:{output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables = {"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)
final_res = parser.invoke(res)

In [72]:
print(res.content)

{"name":"이순신","yob":1545,"yod":1598,"profile":"조선 중기의 무신이자 해군 장군으로 임진왜란(1592–1598) 동안 탁월한 해전 지휘로 일본의 해상 보급을 차단하며 다수의 승리를 거두었다. 한산도 대첩(1592) 등 주요 해전에서 거북선 등을 활용해 연전연승을 거두었고, 1598년 노량해전에서 전사했다. 전쟁 중 기록한 난중일기는 그의 전략과 인간적 면모를 보여주는 중요한 사료로 평가된다."}


In [73]:
final_res

{'name': '이순신',
 'yob': 1545,
 'yod': 1598,
 'profile': '조선 중기의 무신이자 해군 장군으로 임진왜란(1592–1598) 동안 탁월한 해전 지휘로 일본의 해상 보급을 차단하며 다수의 승리를 거두었다. 한산도 대첩(1592) 등 주요 해전에서 거북선 등을 활용해 연전연승을 거두었고, 1598년 노량해전에서 전사했다. 전쟁 중 기록한 난중일기는 그의 전략과 인간적 면모를 보여주는 중요한 사료로 평가된다.'}

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [74]:
# JSON 응답 스키마(구조 설계) 정의 -> Pydantic
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")
    
# 변수: key
# Field.description:변수에 넣을 내용
parser = PydanticOutputParser(pydantic_object=PersonInfo)
prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format:{output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables = {"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)
final_res = parser.invoke(res)

In [ ]:
print(type(res.content), type(final_res))

<class 'str'> <class '__main__.PersonInfo'>


In [75]:
print(res.content)

{"name":"이순신 (李舜臣)","yob":1545,"yod":1598,"profile":"조선 중기의 무신이자 해군 장군으로, 임진왜란(1592–1598) 동안 일본 수군을 상대로 탁월한 지휘로 여러 차례 결정적 승리를 거두었다. 거북선 등 전술적·조선술적 혁신과 한산도 대첩(1592), 명량 해전(1597) 등으로 조선의 해상권을 수호했으며, 해전에서 패한 적이 없었다. 전쟁 중의 기록인 『난중일기』를 남겼고, 1598년 노량 해전에서 전사하였다."}


In [77]:
print(final_res)

name='이순신 (李舜臣)' yob=1545 yod=1598 profile='조선 중기의 무신이자 해군 장군으로, 임진왜란(1592–1598) 동안 일본 수군을 상대로 탁월한 지휘로 여러 차례 결정적 승리를 거두었다. 거북선 등 전술적·조선술적 혁신과 한산도 대첩(1592), 명량 해전(1597) 등으로 조선의 해상권을 수호했으며, 해전에서 패한 적이 없었다. 전쟁 중의 기록인 『난중일기』를 남겼고, 1598년 노량 해전에서 전사하였다.'


In [ ]:
final_res.name
final_res.profile

'조선 중기의 무신이자 해군 장군으로, 임진왜란(1592–1598) 동안 일본 수군을 상대로 탁월한 지휘로 여러 차례 결정적 승리를 거두었다. 거북선 등 전술적·조선술적 혁신과 한산도 대첩(1592), 명량 해전(1597) 등으로 조선의 해상권을 수호했으며, 해전에서 패한 적이 없었다. 전쟁 중의 기록인 『난중일기』를 남겼고, 1598년 노량 해전에서 전사하였다.'

# LLM모델에 출력 형식을 설정

- ChatModel객체의 `with_structured_output(pydantic.BaseModel)` 을 이용해 모델의 출력 형식을 모델 자체에 추가할 수있다.
- `OutputParser`는 모델의 출력 결과를 받아서 형식을 변경해 준다. 그래서 Chain에 탈/부착을 통해 형식을 적용하거나 적용하지 않는 것을 자유롭게 할 수있다.
- 모델의 출력 결과를 항상 일정하게 할 경우에는 아예 **모델에 출력 형식을 설정할 수 있다.**

In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
    
# 출력 schema 정의
class OutputSchema(BaseModel):
    is_real_person: bool = Field(..., 
                                description="요청한 인물이 실존인물인지 여부. True:실존인물, False:실존인물이 아님")
    description: str = Field(..., description="요청한 인물에 대한 설명")
model = ChatOpenAI(model='gpt-5-mini')
res = model.invoke("이순신 장군에 대해 설명해줘.")

In [89]:
# 모델에 응답 스키마를 추가.
model_with_output = model.with_structured_output(OutputSchema)

res2 = model_with_output.invoke("이순신 장군에 대해 설명해줘")

In [90]:
print(type(res2))
print(res2)
print(res2.is_real_person)
print(res2.description)

<class '__main__.OutputSchema'>
is_real_person=True description='이순신(1545–1598)은 조선 중기의 해군 장군으로, 임진왜란(1592–1598) 때 조선 수군을 이끌어 일본의 해상 보급로를 차단하고 여러 결정적 승리를 거둔 인물입니다. 한산도 대첩 등에서 학익진(학 모양의 진형)과 기습 전술을 활용해 일본함대를 격파했으며, 명성이 높은 거북선(거북선의 사용 여부와 범위에 관한 학술적 논의는 있으나 전승에서는 거북선이 그의 상징으로 알려짐)을 활용한 전공으로 유명합니다. 1597년 명량 해전에서는 13척의 배로 열세한 조건에서 대승을 거두었고, 1598년 노량해전에서 전사할 때까지 활약했습니다. 그는 정교한 지휘 능력과 전략·전술로 높은 평가를 받으며, 난중일기(戰時日記)는 그의 전쟁 기록과 인간적 면모를 알려 주는 중요한 사료입니다. 사후 충무공(忠武公)이라는 충무의 칭호를 받았고, 한국에서는 국가적 영웅으로 널리 숭앙됩니다.'
True
이순신(1545–1598)은 조선 중기의 해군 장군으로, 임진왜란(1592–1598) 때 조선 수군을 이끌어 일본의 해상 보급로를 차단하고 여러 결정적 승리를 거둔 인물입니다. 한산도 대첩 등에서 학익진(학 모양의 진형)과 기습 전술을 활용해 일본함대를 격파했으며, 명성이 높은 거북선(거북선의 사용 여부와 범위에 관한 학술적 논의는 있으나 전승에서는 거북선이 그의 상징으로 알려짐)을 활용한 전공으로 유명합니다. 1597년 명량 해전에서는 13척의 배로 열세한 조건에서 대승을 거두었고, 1598년 노량해전에서 전사할 때까지 활약했습니다. 그는 정교한 지휘 능력과 전략·전술로 높은 평가를 받으며, 난중일기(戰時日記)는 그의 전쟁 기록과 인간적 면모를 알려 주는 중요한 사료입니다. 사후 충무공(忠武公)이라는 충무의 칭호를 받았고, 한국에서는 국가적 영웅으로 널리 숭앙됩니다.


In [92]:
res3 = model_with_output.invoke("홍길동에 대해 설명해줘.")
print(res3)

is_real_person=False description='홍길동은 조선 시대 소설 『홍길동전』의 주인공으로, 작품은 대체로 허균(1569–1618)에게서 비롯된 것으로 전해진다. 그는 서자로 태어나 신분 제약과 부조리에 맞서 비범한 재능(일부 판본에서는 도술도 소유)으로 활약하는 인물로 그려지며, 부패한 관리들을 약탈해 민중을 도우는 의적 행위와 결국 자신의 나라를 세워 다스리는 이야기가 전개된다. 이 작품은 신분제와 권력 구조에 대한 비판을 담고 있어 한국 문학사에서 중요한 위치를 차지하며, 한국 최초의 한글 소설 가운데 하나로 평가된다. 홍길동은 이후 영화·드라마·만화 등으로 여러 차례 각색되었고, 한국에서 익명이나 가상의 인물을 지칭할 때 쓰이는 대표적 이름이 되었다.'


In [93]:
res4 = model.invoke("홍길동에 대해 설명해줘.")
res4

AIMessage(content='홍길동(洪吉童)은 한국의 대표적인 민중 영웅이자 문학 속 인물입니다. 보통 다음 내용으로 요약됩니다.\n\n- 작품과 출처: 홍길동은 소설 「홍길동전」의 주인공으로, 조선 후기 작품으로 널리 알려져 있습니다. 전통적으로 허균(許筠)을 저자로 보기도 하지만 저자와 창작 시기에 대해서는 학계에서 논쟁이 있습니다. 흔히 한글로 된 초기 소설 가운데 하나, 또는 한국 최초의 소설로 꼽히기도 합니다.\n- 줄거리(간단히): 홍길동은 서얼(서자) 출신으로서 양반 사회의 차별을 받자 집을 떠납니다. 그는 뛰어난 재주와 초능력(혹은 신묘한 기술)을 지니고 도적을 이끌어 부패한 관료와 권세가의 재물을 약탈해 가난한 사람들에게 나눠줍니다. 이후 외국으로 가서 큰 세력을 이루거나 왕이 되는 등 권력을 잡는 결말로 그려지는 경우가 많습니다.\n- 주제와 의의: 신분제·양반 중심 사회의 불공정과 사회적 차별에 대한 비판, 정의 구현(부자의 재산을 약탈해 가난한 사람을 돕는 이야기), 개인의 자아 실현 등이 중심 주제입니다. 민중적 관점과 현실 비판 의식 때문에 한국 문학사와 민중문화에서 중요한 위치를 차지합니다.\n- 현대적 영향: 홍길동은 영화·드라마·만화·뮤지컬 등으로 여러 차례 각색되었고, 한국에서는 예시용 이름(신분증·서식의 샘플 이름)으로도 자주 쓰입니다. 사회적 상징으로서도 널리 인용됩니다.\n\n원하시면 「홍길동전」의 자세한 줄거리, 주요 장면 요약, 허균과의 관련성에 대한 학술적 논쟁, 또는 현대 각색 사례(영화·드라마 등)를 더 자세히 정리해 드릴게요. 어느 쪽을 원하시나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1191, 'prompt_tokens': 15, 'total_tokens': 1206, 'completion_tokens_details': {'accepted_prediction_tokens':

# Streaming 방식 응답 처리

- Streaming 방식 응답 처리란, LLM이 텍스트를 모두 생성할 때까지 기다리지 않고, 생성되는 즉시 **부분적인 결과**를 실시간으로 전달받아 처리하는 방식을 의미한다. 이는 사용자가 응답을 더 빠르게 인지할 수 있게 해 주며, 특히 대화형 서비스, 실시간 UI 출력, 긴 문서 생성과 같은 상황에서 매우 유용하게 활용된다.

- `invoke()` 요청으로 받는 응답은 **비 스트리밍 방식**으로 모든 응답 텍스트 생성이 완료된 이후 그 결과를 한 번에 반환하는 구조이다. 반면 Streaming 방식은 **토큰(token) 단위 또는 여러 토큰이 묶인 청크(chunk) 단위**로 연속적인 데이터 스트림을 전송한다는 점에서 큰 차이가 있다. 즉, Streaming은 마치 사람이 타이핑을 치듯이 응답을 실시간으로 “흘려보내는” 방식이라고 이해할 수 있다.

- `모델.invoke(input, config)` → 응답 데이터
    - 모델이 전체 응답을 모두 생성한 뒤, 최종 결과를 한 번에 반환하는 방식이다.
    - 배치 처리나 후처리가 중요한 경우에 적합하다.
- `모델.stream(input, config)` → Iterator
    - 모델이 토큰을 생성하는 즉시, 순차적으로 결과를 제공하는 Iterator 형태로 반환한다.
    - 실시간 출력, 대화형 인터페이스, 웹 스트리밍 등에 특히 적합하다.

In [87]:
model = ChatOpenAI(model="gpt-5-mini") #, streaming=True)

iterator = model.stream("서울의 유명한 관광지를 소개해줘. 추천 이유도 알려줘")
print(type(iterator))
for token in iterator:
    print(token.content)

<class 'generator'>

아
래
는
 서울
의
 대표
 관광
지
들
(
장
소
별
 짧
은
 소개
 +
 추천
 이유
)
입니다
.
 일정
이나
 관심
사
(
역
사
·
음
식
·
쇼
핑
·
야
경
 등
)를
 알려
주
시면
 더
 맞
춤
 추천
 드
릴
게
요
.


1
.
 경
복
궁
  

-
 소개
:
 조
선
시
대
의
 중심
 궁
궐
.
 근
정
전
,
 경
회
루
 등
 주요
 건
물이
 잘
 보
존
되어
 있음
.
  

-
 추천
 이유
:
 한국
 전
통
 건
축
과
 역사
 이해
에
 최
적
.
 한
복
을
 입
고
 사진
 찍
기
 좋은
 장소
(
한
복
 착
용
 시
 일부
 궁
은
 무료
 입
장
).


2
.
 창
덕
궁
·
후
원
  

-
 소개
:
 자연
과
 조
화를
 이
룬
 궁
궐
로
 후
원
(
비
원
)은
 특히
 아름
다
움
으로
 유명
.
  

-
 추천
 이유
:
 궁
궐
 중
에서도
 숲
과
 정
원이
 뛰
어나
 산
책
·
사진
 촬영
에
 좋
음
.
 후
원
은
 사
전
 예약
 투
어
 권
장
.


3
.
 북
촌
 한
옥
마
을
  

-
 소개
:
 전
통
 한
옥
이
 모
여
있는
 골
목
길
.
 공
방
,
 카
페
,
 갤
러
리
도
 있음
.
  

-
 추천
 이유
:
 전
통
가
옥
과
 골
목
 풍
경
을
 즐
기
며
 여
유
롭게
 산
책
하기
 좋
음
.


4
.
 인
사
동
  

-
 소개
:
 전
통
 공
예
품
,
 갤
러
리
,
 찻
집
이
 모
인
 거리
.
  

-
 추천
 이유
:
 기
념
품
·
전
통
차
·
한국
 공
예
품
 쇼
핑
에
 최
적
.
 전
통
문화
 체
험
 가능
.


5
.
 명
동
  

-
 소개
:
 쇼
핑
과
 길
거리
 음식
의
 중심
지
.
 화
장
품
과
 패
션
 상
점
이
 밀
집
.
  

-
 추천
 이유
:
 최신
 유
행
 쇼
핑
과
 스트
리
트
푸
드
 경험
,
 밤
에도
 활
기
참
.


6
.
 동

In [ ]:
for token in model.stream("서울의 유명한 관광지를 소개해줘. 추천 이유도 알려줘."):
    print(token.content, end="")

서울의 유명한 관광지를 유형별로 골라 간단한 소개와 함께 추천 이유를 알려드릴게요.

1) 경복궁  
- 소개: 조선의 대표 궁궐로 광화문과 근정전, 경회루 등이 유명합니다.  
- 추천 이유: 한국 전통 건축과 왕실 문화를 한눈에 볼 수 있고, 수문장 교대식 등 볼거리가 많습니다. 한복을 입으면 입장료 할인이나 무료 입장 혜택도 있어 사진 찍기 좋습니다.

2) 창덕궁·후원(비원)  
- 소개: 자연과 조화를 이룬 궁궐로 후원(비원)은 특히 아름다운 비밀정원입니다.  
- 추천 이유: 궁궐 내부와 정원의 조화가 뛰어나고, 가이드 투어로 역사와 공간의 의미를 깊게 이해할 수 있습니다.

3) 북촌한옥마을  
- 소개: 전통 한옥들이 보존된 도심의 골목 마을입니다.  
- 추천 이유: 한국 전통 가옥의 구조와 골목 풍경을 체험하기 좋고, 사진 찍기 좋은 포인트가 많습니다.

4) 인사동  
- 소개: 전통 공예품, 갤러리, 찻집이 모여 있는 문화 거리입니다.  
- 추천 이유: 기념품 쇼핑과 전통 차·음식을 경험할 수 있어 관광객에게 인기입니다.

5) 명동  
- 소개: 쇼핑과 길거리 음식으로 유명한 번화가입니다.  
- 추천 이유: 화장품과 패션 쇼핑, 다양한 길거리 먹거리, 외국인 대상 서비스가 잘 갖춰져 있습니다.

6) 동대문(DDP 포함)  
- 소개: 패션 도매시장과 동대문디자인플라자(DDP)의 현대 건축이 유명합니다.  
- 추천 이유: 밤까지 활발한 쇼핑과 디자인 전시, 특유의 야경과 야시장 분위기를 즐길 수 있습니다.

7) 남산서울타워(N서울타워)  
- 소개: 서울 중심의 전망 타워로 도시 전경과 야경이 아름답습니다.  
- 추천 이유: 케이블카 또는 도보로 오를 수 있고, 전망대에서 서울 전체를 한눈에 볼 수 있어 데이트 코스에 좋습니다.

8) 홍대(홍익대학교 주변)  
- 소개: 젊은 예술·음악·카페 문화의 중심지입니다.  
- 추천 이유: 스트리트 퍼포먼스, 라이브클럽, 독립서점과 감성 카페 등 트렌디한 경험을 할 수 있습니다.

9